In [3]:
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time
import os

In [4]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

client = Elasticsearch(

    # For local development
    #hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)
print(client.info())

{'name': 'instance-0000000001', 'cluster_name': 'd03559a7a3054461a5a720acf26f0c63', 'cluster_uuid': '3MF-MPqZRni8KR1GXooGlg', 'version': {'number': '8.16.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ffe992aa682c1968b5df375b5095b3a21f122bf3', 'build_date': '2024-11-19T16:00:31.793213192Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
# delete model if already downloaded and deployed
try:
    client.ml.delete_trained_model(model_id=".elser_model_2", force=True)
    print("Model deleted successfully, We will proceed with creating one")
except exceptions.NotFoundError:
    print("Model doesn't exist, but We will proceed with creating one")

# Creates the ELSER model configuration. Automatically downloads the model if it doesn't exist.
client.ml.put_trained_model(
    model_id=".elser_model_2", input={"field_names": ["text_field"]}
)

Model deleted successfully, We will proceed with creating one


ObjectApiResponse({'model_id': '.elser_model_2', 'model_type': 'pytorch', 'model_package': {'packaged_model_id': 'elser_model_2', 'model_repository': 'https://ml-models.elastic.co', 'minimum_version': '11.0.0', 'size': 438123914, 'sha256': '2e0450a1c598221a919917cbb05d8672aed6c613c028008fedcd696462c81af0', 'metadata': {}, 'tags': [], 'vocabulary_file': 'elser_model_2.vocab.json'}, 'created_by': 'api_user', 'version': '12.0.0', 'create_time': 1732674425762, 'model_size_bytes': 0, 'estimated_operations': 0, 'license_level': 'platinum', 'description': 'Elastic Learned Sparse EncodeR v2', 'tags': ['elastic'], 'metadata': {}, 'input': {'field_names': ['text_field']}, 'inference_config': {'text_expansion': {'vocabulary': {'index': '.ml-inference-native-000002'}, 'tokenization': {'bert': {'do_lower_case': True, 'with_special_tokens': True, 'max_sequence_length': 512, 'truncate': 'first', 'span': -1}}}}, 'location': {'index': {'name': '.ml-inference-native-000002'}}})

In [6]:
while True:
    status = client.ml.get_trained_models(
        model_id=".elser_model_2", include="definition_status"
    )

    if status["trained_model_configs"][0]["fully_defined"]:
        print("ELSER Model is downloaded and ready to be deployed.")
        break
    else:
        print("ELSER Model is downloaded but not ready to be deployed.")
    time.sleep(5)

ELSER Model is downloaded and ready to be deployed.


In [7]:
# Start trained model deployment if not already deployed
client.ml.start_trained_model_deployment(
    model_id=".elser_model_2", number_of_allocations=1, wait_for="starting"
)

while True:
    status = client.ml.get_trained_models_stats(
        model_id=".elser_model_2",
    )
    if status["trained_model_stats"][0]["deployment_stats"]["state"] == "started":
        print("ELSER Model has been successfully deployed.")
        break
    else:
        print("ELSER Model is currently being deployed.")
    time.sleep(5)

ELSER Model is currently being deployed.
ELSER Model is currently being deployed.
ELSER Model has been successfully deployed.


In [8]:
client.ingest.put_pipeline(
    id="elser-ingest-pipeline",
    description="Ingest pipeline for ELSER",
    processors=[
        {
            "inference": {
                "model_id": ".elser_model_2",
                "input_output": [
                    {"input_field": "Description", "output_field": "plot_embedding"}
                ],
            }
        }
    ],
)

ObjectApiResponse({'acknowledged': True})

In [8]:
print(os.getcwd())

/Users/jiayinzheng/Desktop/data


In [9]:
client.indices.delete(index="elser-example-restaurant", ignore_unavailable=True)


ObjectApiResponse({'acknowledged': True})

In [11]:
# Specify the folder containing your JSON files
folder_path = os.getcwd()

# List to store data from all JSON files
all_data = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):  # Ensure it's a JSON file
        file_path = os.path.join(folder_path, filename)
        
        # Open and read the JSON file
        with open(file_path, 'r') as json_file:
            try:
                data = json.load(json_file)
                all_data.append(data)  # Add the file's data to the list
            except json.JSONDecodeError as e:
                print(f"Error reading {filename}: {e}")
                exit()


# Create the index with a proper mapping
index_name = "elser-example-restaurant"
if not client.indices.exists(index=index_name):
    client.indices.create(
        index=index_name,
        settings={"index": {"default_pipeline": "elser-ingest-pipeline"}},
        mappings={
        "properties": {
            "Description": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot_embedding": {"type": "sparse_vector"},
            }
        },
    )
    print(f"Index `{index_name}` created successfully.")

# Prepare the documents to be indexed
documents = []
for doc in all_data:
    documents.append(
        {
            "_index": "elser-example-restaurant",
            "_source": doc,
        }
    )

# Index documents using helpers.bulk
try:
    helpers.bulk(client, documents)
    print("Done indexing documents into `elser-example-movies` index!")
except Exception as e:
    print(f"Error during bulk indexing: {e}")
    exit()

# Allow some time for indexing to complete
time.sleep(3)

Done indexing documents into `elser-example-movies` index!


In [12]:
mapping = client.indices.get_mapping(index="elser-example-restaurant")
print(mapping)

{'elser-example-restaurant': {'mappings': {'properties': {'Address': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'Description': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'Details': {'properties': {'accessibility': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'amenities': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'atmosphere': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'children': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'crowd': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'dining_options': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'from_the_business': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'highlights': {'type': 'text', 'fields

In [26]:
# Query the index to verify the documents
response = client.search(
    index="elser-example-restaurant",  # Replace with your index name
    size=3,  # Number of documents to retrieve
    query={
        "bool": {
            "must": [
                {"match": {"Description": "food"}},
                {"match": {"Address": "Ann Arbor"}},
                {"range": {"Rating": {"gte": 4.0}}}
            ]
        }
      }  
)

# Print the documents
print("Indexed Documents:")
for hit in response["hits"]["hits"]:
    doc_id = hit["_id"]  # Document ID
    source = hit["_source"]["Title"]  # Document content
    print(f"ID: {doc_id}, Document: {source}")

Indexed Documents:
ID: oCh0a5MBZs0cpqYGalMg, Document: Frita Batidos Ann Arbor
